In [2]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import spearmanr
from time import time
from collections import deque

load_dotenv()

data_path = os.getenv('DATA_PATH')

In [3]:
df = pd.read_excel(data_path, sheet_name='wombi_employees')

# Data Prep

In [4]:
df.columns

Index(['wombus_id', 'birth_continent', 'gender', 'age', 'college_degree',
       'problem_solving_skill', 'technology_skill', 'english_skill',
       'most_recent_income', 'total_jobs', 'shirt_color_preference',
       'customer_exp_preference', 'work_env_preference',
       'personal_growth_preference', 'honest_communication_preference',
       'community_service_preference', 'remote_work_preference',
       'industry_preference', 'score'],
      dtype='object')

In [5]:
# remove unethical columns
illegal = ['birth_continent', 'gender', 'age']
irresponsible = ['shirt_color_preference']
df = df.drop(illegal + irresponsible, axis=1)

In [6]:
df

,wombus_id,college_degree,problem_solving_skill,technology_skill,english_skill,most_recent_income,total_jobs,customer_exp_preference,work_env_preference,personal_growth_preference,honest_communication_preference,community_service_preference,remote_work_preference,industry_preference,score
0,1554,NaN,NaN,31.522899,8.274305,65213.0,NaN,Strongly Disagree,Disagree,Agree,Strongly Agree,Strongly Agree,Hybrid,Renewable Energy,61.06
1,1555,0.0,17.016506,24.785969,9.426865,54693.0,NaN,Strongly Disagree,Strongly Agree,Strongly Agree,Disagree,Strongly Agree,Hybrid,Finance,53.20
2,1556,0.0,NaN,27.225131,5.772313,47381.0,4.0,Agree,Agree,Agree,Neutral,Strongly Agree,On-Site,Finance,70.18
3,1557,0.0,20.723280,25.528884,8.825814,44939.0,NaN,Agree,NaN,Agree,Neutral,Strongly Agree,Hybrid,Tech,58.37
4,1558,0.0,15.862225,31.854325,4.140779,57731.0,NaN,Agree,Strongly Agree,Agree,Neutral,Strongly Agree,Hybrid,Finance,61.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233931,235485,0.0,17.332522,28.914904,7.537856,53862.0,NaN,Strongly Agree,Strongly Disagree,Agree,Neutral,Agree,Hybrid,Higher Education,59.15
233932,235486,0.0,17.754821,27.755353,7.587821,52750.0,4.0,Neutral,Disagree,Strongly Disagree,Neutral,Strongly Agree,Remote,Higher Education,44.12
233933,235487,0.0,16.844175,NaN,5.516678,55147.0,3.0,Agree,Strongly Disagree,Strongly Agree,Strongly Agree,Strongly Agree,Hybrid,Renewable Energy,65.28
233934,235488,0.0,11.799902,27.049392,6.406870,68421.0,3.0,Strongly Agree,Disagree,Agree,Neutral,Disagree,Hybrid,Renewable Energy,61.65


In [7]:
# variable lists
categorical_vars = ['customer_exp_preference', 'work_env_preference',
       'personal_growth_preference', 'honest_communication_preference',
       'community_service_preference']

numerical_vars = ['college_degree', 'problem_solving_skill','technology_skill','english_skill','most_recent_income','total_jobs']

one_hot_vars = ['remote_work_preference', 'industry_preference']

# Missingness EDA

In [8]:
# percent missing for each column
df.isnull().mean() * 100

wombus_id                           0.000000
college_degree                     12.001573
problem_solving_skill               4.000667
technology_skill                    9.005454
english_skill                      16.997811
most_recent_income                  3.001248
total_jobs                         33.005181
customer_exp_preference            12.000291
work_env_preference                 1.000274
personal_growth_preference          2.997401
honest_communication_preference     2.999111
community_service_preference        2.002684
remote_work_preference              8.001761
industry_preference                 0.999846
score                               0.000000
dtype: float64

In [9]:
# percent of rows missing n features
missing_counts = df.isnull().sum(axis=1)

# Calculate the percentage of rows missing at least n features
for i in range(0,9):
    print(f"missing {i}+ features, {((missing_counts >= i).mean() * 100).item()}%")

missing 0+ features, 100.0%
missing 1+ features, 69.62844538677246%
missing 2+ features, 29.261849394706246%
missing 3+ features, 7.649100608713494%
missing 4+ features, 1.3037753915600847%
missing 5+ features, 0.15388824293823952%
missing 6+ features, 0.015388824293823952%
missing 7+ features, 0.0008549346829902196%
missing 8+ features, 0.0%


# SK-learn Pipeline

We will use sci-kit learn's pipeline object to easily handle the incoming "prod" data in its natural form.

In [10]:
# encoders
ordinal_encoder = OrdinalEncoder(
    categories=[['Strongly Disagree', 'Disagree', 'Neutral', 'Agree', 'Strongly Agree'] for _ in categorical_vars],
    handle_unknown='use_encoded_value',
    unknown_value=np.nan
    )


onehot_encoder = OneHotEncoder(
    categories=[['Hybrid', 'On-Site', 'Remote'], ['Renewable Energy', 'Finance', 'Tech', 'Higher Education']],  
    handle_unknown='ignore',  # let null values fall into no encoding column
    sparse_output=False
    )

# pipelines for each variable type
categorical_pipeline = Pipeline(steps=[
    ('ordinal_encoder', ordinal_encoder)
])

onehot_pipeline = Pipeline(steps=[
    ('onehot_encoder', onehot_encoder)
])

# processor combines all pipelines
processor = ColumnTransformer([
    ('categorical', categorical_pipeline, categorical_vars),
    ('onehot', onehot_pipeline, one_hot_vars),
    ('passthrough', 'passthrough', numerical_vars)
],remainder='drop'
)


# Fit ETL Pipeline

In [11]:
X_transformed = processor.fit_transform(df)

onehot_columns = processor.named_transformers_['onehot'].named_steps['onehot_encoder'].get_feature_names_out(one_hot_vars)
transformed_columns = categorical_vars + list(onehot_columns) + numerical_vars

# Create the transformed DataFrame
df_transformed = pd.DataFrame(X_transformed, columns=transformed_columns)
df_transformed.head()

,customer_exp_preference,work_env_preference,personal_growth_preference,honest_communication_preference,community_service_preference,remote_work_preference_Hybrid,remote_work_preference_On-Site,remote_work_preference_Remote,industry_preference_Renewable Energy,industry_preference_Finance,industry_preference_Tech,industry_preference_Higher Education,college_degree,problem_solving_skill,technology_skill,english_skill,most_recent_income,total_jobs
0,0.0,1.0,3.0,4.0,4.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,31.522899,8.274305,65213.0,NaN
1,0.0,4.0,4.0,1.0,4.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,17.016506,24.785969,9.426865,54693.0,NaN
2,3.0,3.0,3.0,2.0,4.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,27.225131,5.772313,47381.0,4.0
3,3.0,NaN,3.0,2.0,4.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,20.723280,25.528884,8.825814,44939.0,NaN
4,3.0,4.0,3.0,2.0,4.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,15.862225,31.854325,4.140779,57731.0,NaN


# Model

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X_transformed, df['score'], test_size=0.2, random_state=42)

In [26]:
# CV
dmatrix = xgb.DMatrix(X_train, y_train)

def rho(predt, dtrain):
    y = dtrain.get_label()
    return ('rho', spearmanr(predt, y).statistic)

res = xgb.cv(
    {'device': 'cuda'},
    dmatrix,
    20,
    nfold=5,
    metrics={"rmse"},
    seed=0,
    callbacks=[
        xgb.callback.EvaluationMonitor(show_stdv=True),
    ],
    custom_metric=rho
)

[0]	train-rmse:6.96509+0.00650	train-rho:0.72515+0.00090	test-rmse:6.96957+0.02873	test-rho:0.72323+0.00198
[1]	train-rmse:6.21464+0.00696	train-rho:0.76788+0.00138	test-rmse:6.22935+0.02397	test-rho:0.76515+0.00255
[2]	train-rmse:5.67526+0.00928	train-rho:0.80034+0.00365	test-rmse:5.69483+0.03402	test-rho:0.79737+0.00249
[3]	train-rmse:5.29894+0.01612	train-rho:0.81712+0.00266	test-rmse:5.32268+0.03308	test-rho:0.81445+0.00273
[4]	train-rmse:4.94013+0.06405	train-rho:0.84078+0.00821	test-rmse:4.96840+0.08042	test-rho:0.83819+0.00729
[5]	train-rmse:4.65783+0.08013	train-rho:0.85569+0.00930	test-rmse:4.69298+0.08863	test-rho:0.85314+0.00832
[6]	train-rmse:4.45986+0.03452	train-rho:0.86388+0.00352	test-rmse:4.50090+0.05367	test-rho:0.86117+0.00249
[7]	train-rmse:4.27256+0.03139	train-rho:0.87264+0.00342	test-rmse:4.31791+0.04614	test-rho:0.86986+0.00375
[8]	train-rmse:4.14788+0.01579	train-rho:0.87683+0.00108	test-rmse:4.19353+0.02937	test-rho:0.87432+0.00088
[9]	train-rmse:4.06731+0.013

In [14]:
# train model on entire training data
model = xgb.XGBRegressor(seed=123, device='cpu')

model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device='cpu', early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [15]:
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {round(mae,2)}")
print(f"Mean Squared Error (MSE): {round(mse, 2)}")
print(f"Root Mean Squared Error (RMSE): {round(rmse, 2)}")
print(f"R² Score: {round(r2, 2)}")
print(f'Spearman Correlation: {round(spearmanr(y_pred, y_test).statistic, 2)}')

Mean Absolute Error (MAE): 2.48
Mean Squared Error (MSE): 13.98
Root Mean Squared Error (RMSE): 3.74
R² Score: 0.79
Spearman Correlation: 0.89


# Predict on Incoming Data

In [16]:
incoming_df = pd.read_csv('../../artifacts/data/wombi_candidates 1.csv')
incoming_df['most_recent_income'] = incoming_df['most_recent_income'].astype(str).apply(lambda x: x.replace(',', ''))
preds = model.predict(processor.transform(incoming_df))

In [24]:
prod_preds = pd.read_csv('../../artifacts/data/wombi_candidates 1.csv')
prod_preds['score'] = preds
prod_preds.to_csv('../../artifacts/data/scored_candidates.csv', index=False)

In [17]:
pd.DataFrame(preds, columns=['predicted_score']).describe()

,predicted_score
count,449.000000
mean,58.539413
std,7.582246
min,36.044048
25%,53.741001
50%,58.674244
75%,63.824711
max,77.968674


In [18]:
pd.DataFrame(y_train).describe()

,score
count,187148.000000
mean,58.228545
std,8.142994
min,1.000000
25%,52.770000
50%,58.170000
75%,63.640000
max,100.000000


# Production Load Estimates

In [19]:
# 1 mil requests / 24 hours / 60 mins / 60 s *  8 hours = requests/s during 8 hour workday
print(f'{1_000_000 / 24 / 60 / 60 * 8} requests/s')

92.5925925925926 requests/s


In [20]:
# run experiment 30 times and take average
results = deque()
for _ in range(30):
    start = time()
    end = start + 1
    ii = 0
    # do sklearn processor & xgb predict in sequence for 1s
    while time() < end:
        x = incoming_df.sample(1)
        model.predict(processor.transform(x))
        ii += 1
    results.append(ii)
print(f'{np.mean(results)}/s')

329.73333333333335/s


# Save Model

In [21]:
from pickle import dump

# todo save something human-readable for better testing and visibility
with open('../../artifacts/models/data_ingestion.pkl', 'wb') as f:
    dump(processor, f, protocol=5)

In [22]:
model.save_model('../../artifacts/models/xgboost_model.json')